In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
db = DFDB('../trial/randomforest.pkl', auto_commit=False)

In [9]:
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [10]:
df_trial[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[0:38]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-11 14:48:38.289806,1071,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.606209,0.001154,2.254551,0.465856,1.648342,None
1,2019-05-11 16:28:02.970710,200,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.562026,0.000901,2.257117,0.452502,1.695091,None
2,2019-05-11 16:35:31.980435,190,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.558030,0.000921,2.250631,0.455314,1.692600,None
3,2019-05-11 16:42:51.099131,180,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.556696,0.000909,2.255689,0.459793,1.698994,None
4,2019-05-11 16:51:36.494608,170,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.551708,0.000874,2.248555,0.448692,1.696847,None
5,2019-05-11 17:01:36.317578,160,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.550463,0.000851,2.260096,0.449134,1.709633,None
6,2019-05-11 17:09:37.580623,150,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.547421,0.000996,2.248746,0.454676,1.701324,None
7,2019-05-11 17:18:15.631490,140,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.544545,0.000867,2.245204,0.452453,1.700659,None
8,2019-05-11 17:26:01.767176,130,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.541476,0.000847,2.250093,0.433307,1.708618,None
9,2019-05-11 17:33:10.404806,120,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,0.538778,0.000820,2.255671,0.450223,1.716894,None


In [ ]:
param_idx = 38
column_idx = 20
db_ = db
df_trial_ = df_trial
mytrial = []
columns = copy.deepcopy(df_trial_.loc[column_idx]['param']['columns'])
param = {'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'q05_roll_std_1000',
  'abs_q25_5',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'median__roll_std',
  'min_roll_std_100',
  'max_to_min',
  'iqr_6'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'RandomForestRegressor',
  'init': {'n_estimators': 617,
   'max_depth': 9,
   'max_features': 0.6118189949437737,
   'min_samples_leaf': 0.10163450193406814,
   'random_state': 6370},
  'fit': {}}}
# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
# param['scaler']['init'] = {}
# param['algorithm']['fit'] = {}

selected_columns = EP.revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=[columns[0]], limit=20, remark='start from top1 column 2th')
print(len(selected_columns))
for trial_i in mytrial:
    db_.insert(trial_i)
df_trial_ = db_.select()
df_trial_['kfold'] = df_trial_['param'].apply(lambda x: x['kfold'])

In [12]:
db_.commit()

In [18]:
df_trial_.loc[440]['param']

{'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'q05_roll_std_1000',
  'abs_q25_5',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'median__roll_std',
  'min_roll_std_100',
  'max_to_min',
  'iqr_6'],
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler', 'init': {}},
 'algorithm': {'cls': 'RandomForestRegressor',
  'init': {'n_estimators': 617,
   'max_depth': 9,
   'max_features': 0.6118189949437737,
   'min_samples_leaf': 0.10163450193406814,
   'random_state': 6370},
  'fit': {}}}

In [15]:
df_trial_[['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-11 14:48:38.289806,None,1071,0.606209,1.153520e-03,2.254551,0.465856,1.648342e+00
1,2019-05-11 16:28:02.970710,None,200,0.562026,9.007728e-04,2.257117,0.452502,1.695091e+00
2,2019-05-11 16:35:31.980435,None,190,0.558030,9.206453e-04,2.250631,0.455314,1.692600e+00
3,2019-05-11 16:42:51.099131,None,180,0.556696,9.092509e-04,2.255689,0.459793,1.698994e+00
4,2019-05-11 16:51:36.494608,None,170,0.551708,8.736515e-04,2.248555,0.448692,1.696847e+00
5,2019-05-11 17:01:36.317578,None,160,0.550463,8.505004e-04,2.260096,0.449134,1.709633e+00
6,2019-05-11 17:09:37.580623,None,150,0.547421,9.963601e-04,2.248746,0.454676,1.701324e+00
7,2019-05-11 17:18:15.631490,None,140,0.544545,8.671811e-04,2.245204,0.452453,1.700659e+00
8,2019-05-11 17:26:01.767176,None,130,0.541476,8.473103e-04,2.250093,0.433307,1.708618e+00
9,2019-05-11 17:33:10.404806,None,120,0.538778,8.201284e-04,2.255671,0.450223,1.716894e+00


In [23]:
mytrial =[]
columns_ = selected_columns

def objective(trial):
        
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    max_depth = trial.suggest_int('max_depth', 5, 16)
    max_features = trial.suggest_uniform('max_features', .6, 1)
    min_samples_leaf = trial.suggest_uniform('min_samples_leaf', 0.1, 0.5)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns_,
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'RandomForestRegressor',
            'init':{
                "n_estimators":n_estimators,
                "max_depth":max_depth,
                "max_features":max_features,
                "min_samples_leaf":min_samples_leaf,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 40')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-18 08:18:19,176] Finished trial#0 resulted in value: 0.15203091377074493. Current best value is 0.15203091377074493 with parameters: {'n_estimators': 389, 'max_depth': 7, 'max_features': 0.9412196272301738, 'min_samples_leaf': 0.11858838926625759, 'random_state': 1344}.
[I 2019-05-18 08:20:40,556] Finished trial#1 resulted in value: 0.14742018273935079. Current best value is 0.14742018273935079 with parameters: {'n_estimators': 993, 'max_depth': 6, 'max_features': 0.8663215564148644, 'min_samples_leaf': 0.11735110329998513, 'random_state': 1677}.
[I 2019-05-18 08:20:42,613] Finished trial#2 resulted in value: 0.0021601955197078997. Current best value is 0.0021601955197078997 with parameters: {'n_estimators': 370, 'max_depth': 9, 'max_features': 0.7870536516140948, 'min_samples_leaf': 0.4081619047795777, 'random_state': 1368}.
[I 2019-05-18 08:21:24,441] Finished trial#3 resulted in value: 0.03448681436415998. Current best value is 0.0021601955197078997 with parameters: {'n_e

[I 2019-05-18 08:37:06,410] Finished trial#56 resulted in value: 0.002163229963808081. Current best value is 0.002150079549674844 with parameters: {'n_estimators': 686, 'max_depth': 13, 'max_features': 0.9032622664763919, 'min_samples_leaf': 0.49113089565809787, 'random_state': 9212}.
[I 2019-05-18 08:37:09,275] Finished trial#57 resulted in value: 0.00214538447047399. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:37:11,169] Finished trial#58 resulted in value: 0.002171335971008242. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:38:05,661] Finished trial#59 resulted in value: 0.1279500759099678. Current best value is 0.00214538447047399 with parameters: {'n

[I 2019-05-18 08:48:06,123] Finished trial#112 resulted in value: 0.027524406192789813. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:48:47,397] Finished trial#113 resulted in value: 0.035852775897868984. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:48:52,090] Finished trial#114 resulted in value: 0.002167433003920818. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:49:27,369] Finished trial#115 resulted in value: 0.037340253173635624. Current best value is 0.00214538447047399 with parameters

[I 2019-05-18 08:59:10,784] Finished trial#168 resulted in value: 0.0371169461189405. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:59:14,325] Finished trial#169 resulted in value: 0.002164713250872238. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:59:17,323] Finished trial#170 resulted in value: 0.0021636783483817133. Current best value is 0.00214538447047399 with parameters: {'n_estimators': 513, 'max_depth': 15, 'max_features': 0.8827083655650999, 'min_samples_leaf': 0.4582789811149415, 'random_state': 2005}.
[I 2019-05-18 08:59:21,217] Finished trial#171 resulted in value: 0.00217020374789437. Current best value is 0.00214538447047399 with parameters: 

In [24]:
for trial_i in mytrial:
    db.insert(trial_i)

In [29]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 40')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
78,2019-05-18 08:31:09.037848,tune 40,21,2.111850,0.002311,2.145305,0.007731,0.033456
88,2019-05-18 08:35:01.838405,tune 40,21,2.159692,0.001855,2.182495,0.010075,0.022804
174,2019-05-18 08:52:09.149143,tune 40,21,2.193001,0.001675,2.210125,0.009827,0.017123
96,2019-05-18 08:37:00.251487,tune 40,21,2.232539,0.002243,2.245187,0.007521,0.012648
45,2019-05-18 08:21:24.439448,tune 40,21,2.232313,0.002082,2.247657,0.007442,0.015343
48,2019-05-18 08:22:16.729684,tune 40,21,2.232597,0.002142,2.247716,0.007361,0.015119
102,2019-05-18 08:38:48.951491,tune 40,21,2.235607,0.001949,2.252153,0.007751,0.016546
74,2019-05-18 08:28:49.235098,tune 40,21,2.236251,0.002044,2.252390,0.007559,0.016140
172,2019-05-18 08:51:23.373586,tune 40,21,2.236388,0.001999,2.253167,0.007663,0.016778
141,2019-05-18 08:45:38.343988,tune 40,21,2.236790,0.002003,2.253397,0.007745,0.016607


In [27]:
db.commit()